# Capítulo 07 — Instruction Tuning: Criando um Assistente

🎯 **Objetivos:** Transformar o modelo completador em um assistente útil usando **SFT (Supervised Fine-Tuning)**.

![SFT](./infograficos/04-pipeline-sft.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
# ============================================================
# 1. Setup e Conexão com a Inteligência Base
# ============================================================
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 O arquivo 'gpt_checkpoint.pt' não foi encontrado.")
    print("Por favor, suba o checkpoint gerado no final do Capítulo 05:")
    uploaded = files.upload()

try:
    ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
    stoi, itos = ckpt['stoi'], ckpt['itos']
    config = ckpt['config']
    context_size = config.context_size
    print(f"✅ Inteligência Base Carregada! Vocabulário: {len(stoi)} caracteres.")
    print(f"📏 Contexto: {context_size}")
except Exception as e:
    print(f"❌ ERRO AO CARREGAR: {e}")
    raise

# Encoder alinhado ao Cap 05 (mapeia desconhecidos para espaço)
def encode(s):
    unk_id = stoi.get(' ', 0)
    return [stoi.get(c, unk_id) for c in s.lower()]

decode = lambda l: ''.join([itos[i] for i in l])


## 1. Dataset de Instruções

Criamos pares de Pergunta e Resposta para o alinhamento.

In [ ]:
instructions = [
    {"q": "o que o gato fez?", "a": "o gato subiu no telhado e pulou o muro."},
    {"q": "onde o cachorro dormiu?", "a": "o cachorro dormiu no sofa e no tapete."},
    {"q": "defina inteligencia artificial", "a": "inteligencia artificial e o estudo de algoritmos."},
    {"q": "o que e machine learning?", "a": "machine learning permite que sistemas aprendam padroes."}
]

PROMPT_PREFIX = "### comando:\n"
PROMPT_SUFFIX = "### resposta:\n"

def build_sft_dataset(data, context_size):
    X, Y, masks = [], [], []
    pad_id = stoi.get(' ', 0)
    for item in data:
        cmd = f"{PROMPT_PREFIX}{item['q']}\n{PROMPT_SUFFIX}"
        full = cmd + item['a'] + "\n"
        cmd_ids = encode(cmd)
        full_ids = encode(full)
        if len(full_ids) < 2:
            continue
        # Mantém o começo para preservar o prompt
        if len(full_ids) > context_size + 1:
            full_ids = full_ids[: context_size + 1]
        cmd_len = min(len(cmd_ids), len(full_ids))
        x = full_ids[:-1]
        y = full_ids[1:]
        if cmd_len >= len(x):
            continue
        m = [1 if (i + 1) >= cmd_len else 0 for i in range(len(x))]
        if len(x) < context_size:
            pad_len = context_size - len(x)
            x = x + [pad_id] * pad_len
            y = y + [pad_id] * pad_len
            m = m + [0] * pad_len
        X.append(x); Y.append(y); masks.append(m)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(masks).to(device)

X, Y, M = build_sft_dataset(instructions, context_size)
print(f"📦 Amostras de Alinhamento: {len(X)}")
# Debug de comprimentos
for item in instructions:
    cmd = f"{PROMPT_PREFIX}{item['q']}\n{PROMPT_SUFFIX}"
    full = cmd + item['a'] + "\n"
    print('---')
    print('q:', item['q'])
    print('cmd_len:', len(encode(cmd)), 'full_len:', len(encode(full)))


## 2. Treinamento com Máscara de Loss

Otimizamos apenas a geração da resposta.

![Masking](./infograficos/03-mascaramento-loss-resposta.png)

In [ ]:
model = GPTMini(config).to(device)
model.load_state_dict(ckpt['state_dict'])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

loss_history = []
model.train()
print("🔨 Alinhando o assistente...")
if len(X) == 0:
    raise RuntimeError("Dataset vazio. Re-treine o Capítulo 05 com context_size maior e recarregue o checkpoint.")
batch_size = min(4, len(X))
for step in range(401):
    idx = torch.randint(len(X), (batch_size,))
    logits, _ = model(X[idx])
    B, T, V = logits.shape
    loss = F.cross_entropy(logits.view(-1, V), Y[idx].view(-1), reduction='none')
    mask = M[idx].view(-1)
    loss = (loss * mask).sum() / mask.sum().clamp(min=1)
    optimizer.zero_grad(set_to_none=True); loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    loss_history.append(loss.item())
    if step % 100 == 0: print(f"Step {step} | Loss {loss.item():.4f}")

plt.plot(loss_history, color='#34A853')
plt.title("Curva de Alinhamento (SFT)")
plt.show()


## 3. Teste do Assistente Alinhado

O modelo agora responde apenas o que foi solicitado.

In [ ]:
@torch.no_grad()
def ask(model, question, max_new_tokens=80):
    model.eval()
    prompt = f"{PROMPT_PREFIX}{question}\n{PROMPT_SUFFIX}"
    idx = torch.tensor(encode(prompt)).unsqueeze(0).to(device)
    prompt_len = idx.shape[1]

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        logits, _ = model(idx_cond)
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
        if itos[next_id.item()] == '\n':
            break

    return decode(idx[0][prompt_len:].tolist())

print("🤖 TESTE DE INTERAÇÃO:")
print("-" * 30)
q1 = "o que o gato fez?"
print(f"Pergunta: {q1}\nResposta: {ask(model, q1)}")

print("\n" + "-" * 30)
q2 = "o que e machine learning?"
print(f"Pergunta: {q2}\nResposta: {ask(model, q2)}")

print("\n" + "-" * 30)
q3 = "defina inteligencia artificial"
print(f"Pergunta: {q3}\nResposta: {ask(model, q3)}")

print("\n" + "-" * 30)
q4 = "onde o cachorro dormiu?"
print(f"Pergunta: {q4}\nResposta: {ask(model, q4)}")


## 🏁 Conclusão da Jornada

Você completou a série! 

Transformou um modelo estatístico em um assistente capaz de seguir intenções humanas. Este é o fundamento do alinhamento de IA.

![Avaliação](./infograficos/05-avaliacao-respostas.png)